In [3]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 5, Finished, Available, Finished)

In [4]:
df_sales= pd.read_excel("abfss://ffba5c1d-6c17-4857-92b8-283135916ea3@onelake.dfs.fabric.microsoft.com/2f70d0d4-b2dd-4276-a27c-dd99afb07eed/Files/current/Sales*.xlsx",sheet_name="Sales")

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 6, Finished, Available, Finished)

In [5]:
df1=spark.createDataFrame(df_sales)

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 7, Finished, Available, Finished)

In [6]:
display(df1.head(10))

StatementMeta(, 508e8dab-d1b3-4a98-867f-414a0f41274a, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 56561bd5-e30c-47a6-97a5-ef051dec2332)

In [6]:
df_Returns= pd.read_excel("abfss://ffba5c1d-6c17-4857-92b8-283135916ea3@onelake.dfs.fabric.microsoft.com/2f70d0d4-b2dd-4276-a27c-dd99afb07eed/Files/current/Sales*.xlsx",sheet_name="Returns")

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 8, Finished, Available, Finished)

In [7]:
df2=spark.createDataFrame(df_Returns)

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 9, Finished, Available, Finished)

In [8]:
display(df2.head(10))

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2c724f9a-9fec-45b3-9d1f-e19c05b46c02)

In [9]:
df_final=df1.join(df2,df1.Order_ID==df2.Order_ID,how="left").drop(df2.Order_ID,df2.Customer_Name,df2.Sales_Amount)

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 11, Finished, Available, Finished)

In [10]:
df_mod=df_final.withColumns({"Order_Year": year("Order_Date"), \
                "Order_Month": month("Order_Date"), \
                "Created_TS": current_timestamp(),\
                "Modified_TS": current_timestamp(),\
                })

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 12, Finished, Available, Finished)

In [11]:
df_mod.createOrReplaceTempView("ViewSales")

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 13, Finished, Available, Finished)

In [12]:
%%sql
create table if not exists Project_Lakehouse.Bronze_Sales
(
    Order_ID	string,
    Order_Date	Date,
    Shipping_Date	date,
    Aging	int,
    Ship_Mode	string,
    Product_Category	string,
    Product	 string,
    Sales 	float,
    Quantity	float,
    Discount	 float,
    Profit 	 float,
    Shipping_Cost 	float,
    Order_Priority	string,
    Customer_ID	string,
    Customer_Name	string,
    Segment	string,
    City	string,
    State	string,
    Country	string,
    Region string,
	Return string,
    Order_Year int,
    Order_Month int,
    Created_TS  TIMESTAMP,
    Modified_TS  TIMESTAMP
)
using DELTA
PARTITIONED by(Order_Year,Order_Month)



StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 14, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
%%sql
Merge into  Project_Lakehouse.Bronze_Sales as BS
using ViewSales as VS
on  BS.Order_Year=VS.Order_Year and BS.Order_Month=VS.Order_Month and BS.Order_ID=VS.Order_ID
when matched then 
update SET
BS.Order_Date	=	VS.Order_Date	,
BS.Shipping_Date	=	VS.Shipping_Date	,
BS.Aging	=	VS.Aging	,
BS.Ship_Mode	=	VS.Ship_Mode	,
BS.Product_Category	=	VS.Product_Category	,
BS.Product	=	VS.Product	,
BS.Sales 	=	VS.Sales 	,
BS.Quantity	=	VS.Quantity	,
BS.Discount	=	VS.Discount	,
BS.Profit 	=	VS.Profit 	,
BS.Shipping_Cost 	=	VS.Shipping_Cost 	,
BS.Order_Priority	=	VS.Order_Priority	,
BS.Customer_ID	=	VS.Customer_ID	,
BS.Customer_Name	=	VS.Customer_Name	,
BS.Segment	=	VS.Segment	,
BS.City	=	VS.City	,
BS.State	=	VS.State	,
BS.Country	=	VS.Country	,
BS.Region  	=	VS.Region  	,
BS.Return  	=	VS.Return  	,
BS.Modified_TS	=	VS.Modified_TS	

when not matched then 
INSERT
(
BS.Order_ID,    
BS.Order_Date	,
BS.Shipping_Date	,
BS.Aging	,
BS.Ship_Mode	,
BS.Product_Category	,
BS.Product	,
BS.Sales 	,
BS.Quantity	,
BS.Discount	,
BS.Profit 	,
BS.Shipping_Cost 	,
BS.Order_Priority	,
BS.Customer_ID	,
BS.Customer_Name	,
BS.Segment	,
BS.City	,
BS.State	,
BS.Country	,
BS.Region  	,
BS.Return,
BS.Order_Year  	,
BS.Order_Month  	,
BS.Created_TS   	,
BS.Modified_TS	
)
values
(
VS.Order_ID,
VS.Order_Date	,
VS.Shipping_Date	,
VS.Aging	,
VS.Ship_Mode	,
VS.Product_Category	,
VS.Product	,
VS.Sales 	,
VS.Quantity	,
VS.Discount	,
VS.Profit 	,
VS.Shipping_Cost 	,
VS.Order_Priority	,
VS.Customer_ID	,
VS.Customer_Name	,
VS.Segment	,
VS.City	,
VS.State	,
VS.Country	,
VS.Region  	,
VS.Return,
VS.Order_Year  	,
VS.Order_Month  	,
VS.Created_TS   	,
VS.Modified_TS	

)


StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 15, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>

In [14]:
%%sql
select * from project_lakehouse.bronze_sales limit 10

StatementMeta(, 92437c9b-e1c4-4fb4-b28d-13a5b9891376, 16, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 25 fields>